<a href="https://colab.research.google.com/github/datastax/ragstack-ai/blob/main/examples/notebooks/quickstart.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Quickstart with RAGStack

This notebook demonstrates how to set up a simple RAG pipeline with RAGStack. At the end of this notebook, you will have a fully functioning Question/Answer model that can answer questions using your supplied documents.

A RAG pipeline requires, at minimum, a vector store, an embedding model, and an LLM. In this tutorial, you will use an Astra DB vector store, an OpenAI embedding model, an OpenAI LLM, and LangChain to orchestrate it all together.

## Prerequisites

You will need a vector-enabled Astra database and an OpenAI Account.

* Create an [Astra vector database](https://docs.datastax.com/en/astra-serverless/docs/getting-started/create-db-choices.html).
* Create an [OpenAI account](https://openai.com/)
* Within your database, create an [Astra DB Access Token](https://docs.datastax.com/en/astra-serverless/docs/manage/org/manage-tokens.html) with Database Administrator permissions.
* Get your Astra DB Endpoint:
  * `https://<ASTRA_DB_ID>-<ASTRA_DB_REGION>.apps.astra.datastax.com`

See the [Prerequisites](https://docs.datastax.com/en/ragstack/docs/prerequisites.html) page for more details.

## Setup
`ragstack-ai` includes all the packages you need to build a RAG pipeline.

`datasets` is used to import a sample dataset

In [1]:
! pip install -q ragstack-ai datasets

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
apple-afm-plugin 0.3.5 requires langchain-community<=0.0.15, but you have langchain-community 0.0.38 which is incompatible.


In [2]:
import cassio

In [4]:
import os
from getpass import getpass

# Enter your settings for Astra DB and OpenAI:
os.environ["ASTRA_DB_API_ENDPOINT"] ="https://ccd53****************s-east-2.apps.astra.datastax.com"
os.environ["ASTRA_DB_APPLICATION_TOKEN"] ="AstraCS:tbtLlpiKq************************5a9b6552c2de3520f82ca8e04e9180bfe706b5b3aaf5"
os.environ["OPENAI_API_KEY"] = getpass("Enter your OpenAI API Key: ")

## Create RAG Pipeline

### Embedding Model and Vector Store

In [5]:
from langchain_astradb import AstraDBVectorStore
from langchain.embeddings import OpenAIEmbeddings
import os

# Configure your embedding model and vector store
embedding = OpenAIEmbeddings()
vstore = AstraDBVectorStore(
    collection_name="DATASTAXDEMO",
    embedding=embedding,
    token=os.getenv("ASTRA_DB_APPLICATION_TOKEN"),
    api_endpoint=os.getenv("ASTRA_DB_API_ENDPOINT"),
)
print("Astra vector store configured")

/Users/atul595525/repository1/LLMAPP/llmvenv/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


Astra vector store configured


In [6]:
from datasets import load_dataset

# Load a sample dataset
philo_dataset = load_dataset("datastax/philosopher-quotes")["train"]
print("An example entry:")
print(philo_dataset[16])

/Users/atul595525/repository1/LLMAPP/llmvenv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Generating train split: 100%|██████████| 450/450 [00:00<00:00, 19922.07 examples/s]

An example entry:
{'author': 'aristotle', 'quote': 'Love well, be loved and do something of value.', 'tags': 'love;ethics'}


In [7]:
from langchain.schema import Document

# Constructs a set of documents from your data. Documents can be used as inputs to your vector store.
docs = []
for entry in philo_dataset:
    metadata = {"author": entry["author"]}
    if entry["tags"]:
        # Add metadata tags to the metadata dictionary
        for tag in entry["tags"].split(";"):
            metadata[tag] = "y"
    # Create a LangChain document with the quote and metadata tags
    doc = Document(page_content=entry["quote"], metadata=metadata)
    docs.append(doc)

In [8]:
docs

[Document(page_content="True happiness comes from gaining insight and growing into your best possible self. Otherwise all you're having is immediate gratification pleasure, which is fleeting and doesn't grow you as a person.", metadata={'author': 'aristotle', 'knowledge': 'y'}),
 Document(page_content='The roots of education are bitter, but the fruit is sweet.', metadata={'author': 'aristotle', 'education': 'y', 'knowledge': 'y'}),
 Document(page_content='Before you heal the body you must first heal the mind', metadata={'author': 'aristotle', 'ethics': 'y'}),
 Document(page_content='The proof that you know something is that you are able to teach it', metadata={'author': 'aristotle', 'education': 'y', 'knowledge': 'y'}),
 Document(page_content='Those who are not angry at the things they should be angry at are thought to be fools, and so are those who are not angry in the right way, at the right time, or with the right persons.', metadata={'author': 'aristotle'}),
 Document(page_content=

In [13]:
# Create embeddings by inserting your documents into the vector store.
inserted_ids = vstore.add_documents(docs)
print(f"\nInserted {len(inserted_ids)} documents.")


Inserted 450 documents.


In [11]:
# Checks your collection to verify the documents are embedded.
print(vstore.astra_db.collection("DATASTAXDEMO").find())

{'data': {'documents': [{'_id': 'fb92b356745741298d7f2fcaf4ff5118', 'content': 'Whoever fights monsters should see to it that in the process he does not become a monster. And if you gaze long enough into an abyss, the abyss will gaze back into you.', '$vector': [0.003699381870495875, -0.030650183779861822, 0.022234895007287203, -0.0019654977479864923, -0.014823262717309786, 0.006626718788395979, -0.019507002903536984, -0.010210294661060508, 0.022106219761978296, 0.006459442552742692, 0.019790085835513508, 0.007842689332620624, 0.04287422963873029, -0.018554813166302257, 0.022106219761978296, 0.002515579671937195, 0.045859468507808226, 0.0015746499732725862, 0.01789857537569099, -0.014848997952636072, -0.015929860141212133, 0.005063328155201617, 0.0071028131913520795, -0.035076575338116256, -0.03113914486915854, 0.004529330171253265, 0.019918759218177366, -0.020896683259415856, 0.01706219373176329, -0.017177999962425264, 0.01549236859124546, -0.025027122871046827, -0.003292450039364302,

### Basic Retrieval

Retrieve context from your vector database, and pass it to the model with a prompt.

In [14]:
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough

retriever = vstore.as_retriever(search_kwargs={"k": 3})

prompt_template = """
Answer the question based only on the supplied context. If you don't know the answer, say you don't know the answer.
Context: {context}
Question: {question}
Your answer:
"""
prompt = ChatPromptTemplate.from_template(prompt_template)
model = ChatOpenAI()

chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

chain.invoke("In the given context, what is the most important to allow the brain and provide me the tags?")

/Users/atul595525/repository1/LLMAPP/llmvenv/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


"The most important thing in the given context is to allow the brain the full measure of sleep required to restore it. The tags associated with the context are 'author': 'schopenhauer', 'ethics': 'y', 'knowledge': 'y'."

In [ ]:
# Add your questions here!
# chain.invoke("<your question>")

## Cleanup

In [ ]:
# WARNING: This will delete the collection and all documents in the collection
vstore.delete_collection()

You now have a fully functioning RAG pipeline! Note that there are several different ways to accomplish this, depending on your input data format, vector store, embedding, model, output type, and more. There are also more advanced RAG techniques that leverage new ingestion, retrieval, and generation patterns.  

RAG is a powerful solution used in tandem with the capabilities of LLMs. Check out our other examples for ideas on how you can build innovative solutions using RAGStack!